In [16]:
pip install matplotlib


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from scipy.optimize import minimize
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt



def obj_loss_function(params, X, y, lamb):
    w0 = params[-1]  
    w = params[:-1]  
    w = w[:,None]


    log_loss = np.sum(np.logaddexp(0, (- y * (X @ w + w0))))


    reg_term = 0.5 * lamb * float(w.T @ w)
    total_loss = log_loss + reg_term

    return total_loss


def minBinDev(X, y, lamb):
    n,d = X.shape
    initial_params = np.ones(d + 1) * lamb
    print(initial_params)

    sol = minimize(obj_loss_function, initial_params, args=(X, y, lamb))

    if not sol.success:
        print("Warning: Optimization did not converge. Check your data or parameters.")


    w = sol['x'][:-1][:, None]  # make it d-by-1
    w0 = sol['x'][-1]
    print(w)
    print(w0)

    return w, w0




'X = np.random.randn(100, 5) \ny = np.random.choice([-1, 1], size=100) \nlamb = 0.1  \n\nw_opt, w0_opt = minBinDev(X, y, lamb)\n\nprint("Optimized weights:", w_opt)\nprint("Optimized bias:", w0_opt)'

In [15]:
#q1b

def minHinge(X, y, lamb, stabilizer=1e-5):
    n,d = X.shape

    y = y.flatten()

    #P in (P,q,G,h)
    P = np.zeros((n+d+1,n+d+1))

    #P[:d, :d] = lamb * np.eye(d)  # Regularization on the weights


    #for size of dimensioanlity, create that size identity matrix in the top left
    P[:d, :d] = lamb * np.eye(d)

    #q = d+1 zeroes, n 1's
    q = np.hstack([np.zeros(d+1), np.ones(n)])

    #G matrix: 2n rows for the slack variables for 2 constraints
    G = np.zeros((2*n, d+1+n))

    #first constraint, G11 (could be 0)
    G[:n, :d] = -y[:, None] * X

    #g12 for w0 (could also be 0)
    G[:n,d]= -y

    #g13, Iden matrix (prob good)
    G[:n,d+1:]= -np.eye(n) #mario agrees

    #second contraint 
    G[n:, d+1:] = -np.eye(n) #mario agrees

    #h, constraint for G, so 2n rows to match, 0n and -1n.
    h = np.hstack([-np.ones(n), np.zeros(n)]) 

    P += stabilizer * np.eye(d+1+n)

    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)

    solution = solvers.qp(P,q,G,h)

    w = np.array(solution['x'][:d])
    w0 = np.array(solution['x'][d]).item()

    return w,w0




In [2]:

def classify(Xtest, w, w0):
    return np.sign(Xtest @ w + w0)

In [7]:
def generateData(n, gen_model):

    # Controlling the random seed will give you the same 
    # random numbers every time you generate the data. 
    # The seed controls the internal random number generator (RNG).
    # Different seeds produce different random numbers. 
    # This can be handy if you want reproducible results for debugging.
    # For example, if your code *sometimes* gives you an error, try
    # to find a seed number (0 or others) that produces the error. Then you can
    # debug your code step-by-step because every time you get the same data.

    # np.random.seed(0)  # control randomness when debugging

    if gen_model == 1 or gen_model == 2:
        # Gen 1 & 2
        d = 2
        w_true = np.ones([d, 1])

        X = np.random.randn(n, d)

        if gen_model == 1:
            y = np.sign(X @ w_true)  # generative model 1
        else:
            y = np.sign((X ** 2) @ w_true - 1)  # generative model 2

    elif gen_model == 3:
        # Gen 3
        X, y = generateMoons(n)

    else:
        raise ValueError("Unknown generative model")

    return X, y


def generateMoons(n, noise=0.1):
    n_samples_out = n // 2
    n_samples_in = n - n_samples_out
    outer_circ_x = np.cos(np.linspace(0, np.pi, n_samples_out))
    outer_circ_y = np.sin(np.linspace(0, np.pi, n_samples_out))
    inner_circ_x = 1 - np.cos(np.linspace(0, np.pi, n_samples_in))
    inner_circ_y = 1 - np.sin(np.linspace(0, np.pi, n_samples_in)) - 0.5

    X = np.vstack(
        [np.append(outer_circ_x, inner_circ_x), 
         np.append(outer_circ_y, inner_circ_y)]
    ).T
    X += np.random.randn(*X.shape) * noise

    y = np.hstack(
        [-np.ones(n_samples_out, dtype=np.intp), 
         np.ones(n_samples_in, dtype=np.intp)]
    )[:, None]
    return X, y

In [16]:

#q1d

def compute_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)


def synExperimentsRegularize():

    n_runs = 100 
    n_train = 100  
    n_test = 1000  

    lamb_list = [0.001, 0.01, 0.1, 1.0]  
    gen_model_list = [1, 2, 3]  

    train_acc_bindev = np.zeros((len(lamb_list), len(gen_model_list), n_runs))
    test_acc_bindev = np.zeros((len(lamb_list), len(gen_model_list), n_runs))
    train_acc_hinge = np.zeros((len(lamb_list), len(gen_model_list), n_runs))
    test_acc_hinge = np.zeros((len(lamb_list), len(gen_model_list), n_runs))

    for r in range(n_runs):
        for i, lamb in enumerate(lamb_list):
            for j, gen_model in enumerate(gen_model_list):

                Xtrain, ytrain = generateData(n=n_train, gen_model=gen_model)
                Xtest, ytest = generateData(n=n_test, gen_model=gen_model)

                
                # Binary deviance loss (part a)
                w, w0 = minBinDev(Xtrain, ytrain, lamb)
                ytrain_pred = classify(Xtrain, w, w0)  
                ytest_pred = classify(Xtest, w, w0)  

                train_acc_bindev[i, j, r] = compute_accuracy(ytrain, ytrain_pred)

                train_acc_bindev[i, j, r] = compute_accuracy(ytrain, ytrain_pred)
                test_acc_bindev[i, j, r] = compute_accuracy(ytest, ytest_pred)

                # Hinge loss (part b)
                w, w0 = minHinge(Xtrain, ytrain, lamb)
                ytrain_pred = classify(Xtrain, w, w0)  # Predict on training set
                ytest_pred = classify(Xtest, w, w0)  # Predict on test set
                # Store accuracies
                train_acc_hinge[i, j, r] = compute_accuracy(ytrain, ytrain_pred)
                test_acc_hinge[i, j, r] = compute_accuracy(ytest, ytest_pred)

    # Compute average 
    avg_train_acc_bindev = np.mean(train_acc_bindev, axis=2)
    avg_test_acc_bindev = np.mean(test_acc_bindev, axis=2)
    avg_train_acc_hinge = np.mean(train_acc_hinge, axis=2)
    avg_test_acc_hinge = np.mean(test_acc_hinge, axis=2)

    # Combine accuracies
    train_acc = np.hstack([avg_train_acc_bindev, avg_train_acc_hinge])
    test_acc = np.hstack([avg_test_acc_bindev, avg_test_acc_hinge])

    return train_acc, test_acc


acc1,acc2 = synExperimentsRegularize()

print(acc1)
print(acc2)

[0.001 0.001 0.001]
[[30.31873391]
 [25.50535536]]
-3.2123855437590425
     pcost       dcost       gap    pres   dres
 0: -6.5799e+01  2.7053e+02  2e+03  5e+00  2e+01
 1:  1.2342e+02 -1.2169e+02  3e+02  8e-01  3e+00
 2:  4.0358e+01 -2.0132e+01  7e+01  1e-01  6e-01
 3:  1.2900e+01 -1.7876e+00  2e+01  4e-02  2e-01
 4:  8.0406e+00 -3.6188e-01  1e+01  2e-02  8e-02
 5:  6.3454e+00 -5.3778e-01  8e+00  1e-02  6e-02
 6:  5.4863e+00 -1.2427e+00  8e+00  1e-02  5e-02
 7:  4.5999e+00 -7.6333e-01  6e+00  8e-03  3e-02
 8:  3.2010e+00 -1.5396e+00  5e+00  6e-03  2e-02
 9:  3.4979e+00 -1.4032e+00  5e+00  4e-03  2e-02
10:  2.2658e+00 -7.6177e-01  3e+00  2e-03  9e-03
11:  2.0258e+00 -8.3581e-01  3e+00  2e-03  7e-03
12:  9.9882e-01 -4.3914e-02  1e+00  6e-15  6e-16
13:  6.4382e-01  3.6862e-01  3e-01  7e-15  7e-16
14:  4.6461e-01  4.4479e-01  2e-02  7e-15  4e-16
15:  4.5225e-01  4.5145e-01  8e-04  6e-15  8e-16
16:  4.5173e-01  4.5172e-01  8e-06  6e-15  5e-16
17:  4.5173e-01  4.5173e-01  8e-08  7e-15  6e-16

/var/folders/9p/49ytqgnd2c7cvsxw_9qtt__80000gn/T/ipykernel_28292/2026756684.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  reg_term = 0.5 * lamb * float(w.T @ w)


     pcost       dcost       gap    pres   dres
 0: -6.3519e+01  2.2431e+02  1e+03  4e+00  2e+01
 1:  9.5016e+01 -5.9057e+01  2e+02  6e-01  2e+00
 2:  3.8500e+01 -1.5541e+01  7e+01  2e-01  7e-01
 3:  1.6489e+01 -5.2294e-01  2e+01  6e-02  2e-01
 4:  1.1194e+01  3.7836e+00  9e+00  2e-02  9e-02
 5:  8.7534e+00  5.7611e+00  4e+00  8e-03  3e-02
 6:  7.4456e+00  7.0163e+00  5e-01  4e-04  2e-03
 7:  7.2187e+00  7.1808e+00  4e-02  2e-05  8e-05
 8:  7.2014e+00  7.1952e+00  6e-03  3e-06  1e-05
 9:  7.1981e+00  7.1980e+00  7e-05  3e-08  1e-07
10:  7.1980e+00  7.1980e+00  7e-07  3e-10  1e-09
Optimal solution found.
[0.1 0.1 0.1]
[[-0.04615935]
 [-0.07875349]]
0.08118530345536637
     pcost       dcost       gap    pres   dres
 0: -3.4281e-01  2.0755e+02  4e+02  3e+00  2e+00
 1:  1.4048e+02  8.0968e+01  6e+01  5e-16  1e-15
 2:  1.0031e+02  9.5120e+01  5e+00  3e-16  1e-15
 3:  9.6317e+01  9.5883e+01  4e-01  2e-16  1e-13
 4:  9.6004e+01  9.6000e+01  4e-03  3e-16  5e-14
 5:  9.6001e+01  9.6001e+01  4e